# 12.1 내장 자료형의 상속은 까다롭다

- 사용자가 오버라이드한 코드를 호출하지 않음

In [1]:
class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

In [2]:
dd = DoppelDict(one=1)
dd

{'one': 1}

In [3]:
dd['two'] = 2
dd

{'one': 1, 'two': [2, 2]}

In [4]:
dd.update(three=3)
dd

{'one': 1, 'two': [2, 2], 'three': 3}

In [5]:
class AnswerDict(dict):
    def __getitem__(self, key):
        return 42

In [9]:
ad = AnswerDict(a='foo')
ad['a']

42

In [10]:
d = {}
d.update(ad)
d['a']

'foo'

In [11]:
d

{'a': 'foo'}

- collections.Userdict 상속하면 해결 가능

In [17]:
import collections

In [20]:
class DoppelDict2(collections.UserDict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

In [21]:
dd = DoppelDict2(one=1)
dd

{'one': [1, 1]}

In [22]:
dd.update(three=3)
dd

{'one': [1, 1], 'three': [3, 3]}

In [23]:
class AnswerDict2(collections.UserDict):
    def __getitem__(self, key):
        return 42

In [24]:
ad = AnswerDict2(a='foo')
ad['a']

42

In [25]:
d = {}
d.update(ad)
d['a']
d

{'a': 42}

# 12.2 다중 상속과 메서드 결정 순서

In [65]:
class A:
    def ping(self):
        print('ping:', self)
        
class B(A):
    def pong(self):
        print('pong', self)
    
class C(A):
    def pong(self):
        print('PONG', self)
        
class D(B, C):
    def ping(self):
        super().ping()
        print('post-ping', self)
        
    def pingpong(self):
        self.ping()
        super().ping()
        self.pong()
        super().pong()
        C.pong(self)

In [37]:
a = A()
a.ping()

ping: <__main__.A object at 0x0000025B358351F0>


In [38]:
b = B()
b.ping()

ping: <__main__.B object at 0x0000025B35835DC0>


In [39]:
b.pong()

pong <__main__.B object at 0x0000025B35835DC0>


In [40]:
c = C()
c.pong()

PONG <__main__.C object at 0x0000025B35835160>


In [41]:
c.ping()

ping: <__main__.C object at 0x0000025B35835160>


In [66]:
d = D()
d.pong()

pong <__main__.D object at 0x0000025B35525670>


In [67]:
d.ping()

ping: <__main__.D object at 0x0000025B35525670>
post-ping <__main__.D object at 0x0000025B35525670>


In [68]:
d.pingpong()

ping: <__main__.D object at 0x0000025B35525670>
post-ping <__main__.D object at 0x0000025B35525670>
ping: <__main__.D object at 0x0000025B35525670>
pong <__main__.D object at 0x0000025B35525670>
pong <__main__.D object at 0x0000025B35525670>
PONG <__main__.D object at 0x0000025B35525670>


In [72]:
from diamond import *
d = D()
d.pong()

pong <__main__.D object at 0x0000025B355227C0>


In [73]:
C.pong(d)

PONG <__main__.D object at 0x0000025B355227C0>


In [74]:
D.__mro__

(__main__.D, __main__.B, __main__.C, __main__.A, object)

# 12.3 실세계에서의 다중 상속

- collections.abc 패키지에서 다중 상속을 많이 사용함

- tkinter 모듈에 구현된 Tkinter GUI 툴킷은 표준 라이브러리에서 다중 상속을 극단적으로 사용하는 예를 보여줌 (그림 12-2와 같이)
- Tkinter는 20년이나 되었고, 다중 상속의 단점을 이해하지 못할 때의 예시를 잘 보여줌